In [6]:
import numpy
import pickle
import copy
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [123]:
def combine_results(results,results_smac=None,random_speed_up=False,smac_baseline=False):
    searchers=results[results.keys()[0]].keys()
    n_intervals=len(results[results.keys()[0]]['random'][2])
    if results_smac is None and not random_speed_up:
        return results
    new_results=results
    if results_smac is not None:
        for d in results.keys():
            for s in ['random','nvb']:
                new_results[d][s+'_smac']=results_smac[d][s]
    if smac_baseline:
        #for d in results.keys():
            #trials=[results[d]['smac'][0]]
            #data=[[results[d]['smac'][i][0]]*n_intervals for i in range(1,len(results[d]['smac']))]
            #new_results[d]['rf_baseline']=trials+data
        new_results[d]['rf_baseline']=results[d]['rf_baseline']
    if random_speed_up:
        for d in results.keys():
            for s in ['random','random_smac']:
                if s in new_results[d].keys():
                    for interval in [2,3]:
                        new_data=[data[interval-1::interval] for data in new_results[d][s][1:]]
                        new_data.insert(0,new_results[d][s][0])
                        new_results[d][s+'_'+str(interval)+'x']=new_data
    return new_results
                    

In [116]:
def parse_results(results,is_mean,rf_baseline=False,relative=True):
    searchers=results[results.keys()[0]].keys()
    print searchers
    searchers_val_avg={}
    searchers_test_avg={}
    test_ind = 2
    if is_mean:
        test_ind=3
    if rf_baseline:
        for s in searchers:
            val_errors=numpy.asarray([numpy.asarray(results[d][s][1])-numpy.nanmin(results[d]['rf_baseline'][1]) for d in results.keys()])
            test_errors=numpy.asarray([numpy.asarray(results[d][s][test_ind])-numpy.nanmin(results[d]['rf_baseline'][test_ind]) for d in results.keys()])    
            searchers_val_avg[s]=numpy.mean(val_errors,axis=0)
            searchers_test_avg[s]=numpy.mean(test_errors,axis=0)
    elif relative:
        for s in searchers:
            val_errors=numpy.asarray([(1-numpy.asarray(results[d][s][1]))/(1-numpy.nanmin(results[d]['rf_baseline'][1])) for d in results.keys()])
            test_errors=numpy.asarray([(1-numpy.asarray(results[d][s][test_ind]))/(1-numpy.nanmin(results[d]['rf_baseline'][test_ind])) for d in results.keys()]) 
            searchers_val_avg[s]=numpy.mean(val_errors,axis=0)
            searchers_test_avg[s]=numpy.mean(test_errors,axis=0)
    else:
        for s in searchers:
            val_errors=numpy.asarray([numpy.asarray(results[d][s][1]) for d in results.keys()])
            test_errors=numpy.asarray([numpy.asarray(results[d][s][test_ind]) for d in results.keys()])    
            searchers_val_avg[s]=numpy.mean(val_errors,axis=0)
            searchers_test_avg[s]=numpy.mean(test_errors,axis=0)
    
    return searchers_val_avg,searchers_test_avg

In [3]:
def compute_ranks(results,intervals,is_mean,searchers=None,is_val=False):
    if is_val:
        ind=1
    else:
        if is_mean:
            ind=3
        else:
            ind=2
    all_ranks={}
    if searchers is None:
        searchers=searchers_val_avg.keys()
    for s in searchers:
        all_ranks[s]=[]
    for t in range(len(intervals)):
        t_ranks={}
        for s in searchers:
            t_ranks[s]=[]
        for d in results.keys():
            array=numpy.array([combined_results[d][s][ind][t] for s in searchers])
            ranks=scipy.stats.rankdata(array)
            for s in range(len(searchers)):
                t_ranks[searchers[s]].append(ranks[s])
        for s in searchers:
            all_ranks[s].append(numpy.mean(t_ranks[s]))
    return all_ranks

In [196]:
def chart_errors(searchers_val_avg,searchers_test_avg,searchers_val_avg_rel,searchers_test_avg_rel,test_ranks,val_ranks,is_mean,intervals,file_name=None):
    val_lim=[.08,.13]
    test_lim=[0.1,.15]
    #val_lim=[-0.07,0]
    #test_lim=[-0.05,0]
    val_lim_rel=[0.95,1.05]
    test_lim_rel=[0.95,1.01]
    xlabel='Time (s)'
    if is_mean:
        chart_title='Mean'
        if file_name is None:
            file_name='aggregate_errors_mean.pdf'
    else:
        chart_title='Median'
        if file_name is None:
            file_name='aggregate_errors_median.pdf'
        #val_lim=[0.08,.12]
        #test_lim=[0.09,0.13]
    pdf_file=PdfPages(file_name)
    color={'smac':'r','hyperopt':'b','nvb':'y','random':'g', 'random_2x':'g','random_3x':'g','nvb_smac':'y','random_smac':'g','random_smac_2x':'g','random_smac_3x':'g','rf_baseline':'k'}
    linestyle={'smac':'-','hyperopt':'-','nvb':'-','random':'-', 'random_2x':'-','random_3x':'-','nvb_smac':'--','random_smac':'--','random_smac_2x':'--','random_smac_3x':'--','rf_baseline':'-'}
    marker={'random_2x':'+','random_3x':'x','random_smac_2x':'+','random_smac_3x':'x'}
    #intervals=range(30,3601,30)
    plt.figure(figsize=(11,8.5))
    plt.suptitle(chart_title+ ' Best Validation Error (left) and Corresponding Test Accuracy (right)')
    plt.subplot(121)
    indices=range(4,len(intervals),5)
    for s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']:
        plt.plot([intervals[i] for i in indices],[searchers_val_avg[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in ['random_2x','random_smac_2x']:
        plt.plot([intervals[i] for i in indices],[searchers_val_avg[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.xlabel(xlabel)
    plt.ylim(val_lim)
    plt.legend(loc=1)
    plt.subplot(122)
    for s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']:
        plt.plot([intervals[i] for i in indices],[searchers_test_avg[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in ['random_2x','random_smac_2x']:
        plt.plot([intervals[i] for i in indices],[searchers_test_avg[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.xlabel(xlabel)
    plt.ylim(test_lim)
    plt.legend(loc=1)
    pdf_file.savefig()
    plt.close()
    #Charts relative to rf_baseline
    plt.figure(figsize=(11,8.5))
    plt.suptitle(chart_title+ ' Best Validation Accuracy (left) and Corresponding Test Accuracy (right) Relative to RF Baseline')
    plt.subplot(121)
    indices=range(0,len(intervals),5)
    for s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']:
        plt.plot([intervals[i] for i in indices],[searchers_val_avg_rel[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in ['random_2x','random_smac_2x']:
        plt.plot([intervals[i] for i in indices],[searchers_val_avg_rel[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.xlabel(xlabel)
    plt.ylim(val_lim_rel)
    plt.legend(loc=4)
    plt.subplot(122)
    for s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']:
        plt.plot([intervals[i] for i in indices],[searchers_test_avg_rel[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in ['random_2x','random_smac_2x']:
        plt.plot([intervals[i] for i in indices],[searchers_test_avg_rel[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.xlabel(xlabel)
    plt.ylim(test_lim_rel)
    plt.legend(loc=4)
    pdf_file.savefig()
    plt.close()
    plt.figure(figsize=(11,8.5))
    rank_searchers=val_ranks.keys()
    for s in [s for s in rank_searchers if s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']]:
        plt.plot([intervals[i] for i in indices],[val_ranks[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in [s for s in rank_searchers if s in  ['random_2x','random_smac_2x']]:
        plt.plot([intervals[i] for i in indices],[val_ranks[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.title('Average Rank of '+chart_title+' Val Error Across All Datasets')
    plt.xlabel(xlabel)
    plt.legend(loc=1)
    pdf_file.savefig()
    plt.close()
    plt.figure(figsize=(11,8.5))
    for s in [s for s in rank_searchers if s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']]:
        plt.plot([intervals[i] for i in indices],[test_ranks[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s])
    for s in [s for s in rank_searchers if s in  ['random_2x','random_smac_2x']]:
        plt.plot([intervals[i] for i in indices],[test_ranks[s][i] for i in indices],label=s,color=color[s],linestyle=linestyle[s],marker=marker[s])
    plt.title('Average Rank of '+chart_title+' Test Error Across All Datasets')
    plt.xlabel(xlabel)
    plt.legend(loc=1)
    pdf_file.savefig()
    plt.close()
    plt.figure(figsize=(11,8.5))
    counts={}
    for s in ['nvb_smac','smac','random_smac','hyperopt','nvb','random']:
        counts[s]=[]
        for t in range(len(intervals)):    
            counts[s].append(len([d for d in combined_results.keys() if combined_results[d][s][3][t]<combined_results[d]['rf_baseline'][3][t]]))
    for s in ['nvb_smac','smac','random_smac','hyperopt','nvb','random']:
        plt.plot([intervals[i] for i in indices],[counts[s][i] for i in indices],color=color[s],linestyle=linestyle[s])
    plt.xlabel(xlabel)
    plt.title('Number of Datasets on which Searcher Has Lower Test Error Than RF Baseline')
    plt.legend(loc=4)
    plt.ylim([0,40])
    pdf_file.savefig()
    plt.close()
    pdf_file.close()

In [ ]:
def individual_chart_error(results, is_validation,is_mean,interval_size,searchers=None,filename=None):
    datasets=results.keys()
    if searchers is None:
        searchers=results[datasets[0]].keys()
    color={'smac':'r','hyperopt':'b','nvb':'y','random':'g', 'random_2x':'g','random_3x':'g','nvb_smac':'y','random_smac':'g','random_smac_2x':'g','random_smac_3x':'g','rf_baseline':'k'}
    linestyle={'smac':'-','hyperopt':'-','nvb':'-','random':'-', 'random_2x':'-','random_3x':'-','nvb_smac':'--','random_smac':'--','random_smac_2x':'--','random_smac_3x':'--','rf_baseline':'-'}
    marker={'random_2x':'+','random_3x':'x','random_smac_2x':'+','random_smac_3x':'x'}
    if is_mean:
        test_ind=3
        title1='Mean'
        if filename is None:
            filename1='individual_charts_mean'
    else:
        test_ind=2
        title1='Median'
        if filename is None:
            filename1='individual_charts_median'
    if is_validation:
        ind = 1
        if filename is None:
            filename2='val_error.pdf'
        title2='Best Validation Error for Dataset '
    else:
        ind = test_ind
        if filename is None:
            filename2='test_error.pdf'
        title2='Test Error Corresponding to Best Validation Error for Dataset '
    if filename is None:
        pdf_file=PdfPages(filename1+filename2)
    else:
        pdf_file=PdfPages(filename)
    for d in datasets:
        x = range(interval_size-1,len(results[d]['smac'][ind+1]),interval_size)
        indices=x
        for s in [s for s in searchers if s in ['smac','hyperopt','random','nvb','random_smac','nvb_smac','rf_baseline']]:
            plt.plot(x,[results[d][s][ind][i] for i in indices],color=color[s],linestyle=linestyle[s],label=s)
            #plt.ylim([0,results[d]['rf_baseline'][ind][0]])
        for s in [s for s in searchers if s in ['random_2x','random_smac_2x']]:
            plt.plot(x,[results[d][s][ind][i] for i in indices],color=color[s],marker=marker[s],linestyle=linestyle[s],label=s)
            #plt.ylim([0,results[d]['rf_baseline'][ind][0]])
        plt.title(title1+' '+title2+str(d))
        plt.legend(loc=1)
        pdf_file.savefig()
        plt.close()
    pdf_file.close()
            
        


In [197]:
searchers=['smac','random','nvb','random_smac','nvb_smac','rf_baseline','hyperopt','random_2x','random_smac_2x']
filename='./all_results_condensed.pkl'
results=pickle.load(open(filename,'rb'))
filename='./all_results_condensed_smac_init.pkl'
results_smac=pickle.load(open(filename,'rb'))
intervals=range(30,7201,30)
combined_results=combine_results(results,results_smac,True,True)
searchers_val_avg,searchers_test_avg=parse_results(combined_results,True,False,False)
searchers_val_avg_rel,searchers_test_avg_rel=parse_results(combined_results,True,False,True)
test_ranks=compute_ranks(combined_results,intervals,True,searchers)
val_ranks=compute_ranks(combined_results,intervals,True,searchers,True)
chart_errors(searchers_val_avg,searchers_test_avg,searchers_val_avg_rel,searchers_test_avg_rel,test_ranks,val_ranks,True,intervals,'./aggregate_mean_errors.pdf')
individual_chart_error(combined_results,True,True,5,searchers,'./individual_mean_val_errors.pdf')
individual_chart_error(combined_results,False,True,5,searchers,'./individual_mean_errors.pdf')




['random_3x', 'random_smac_3x', 'hyperopt', 'random', 'nvb_smac', 'random_smac_2x', 'smac', 'nvb', 'random_smac', 'random_2x', 'rf_baseline']
['random_3x', 'random_smac_3x', 'hyperopt', 'random', 'nvb_smac', 'random_smac_2x', 'smac', 'nvb', 'random_smac', 'random_2x', 'rf_baseline']


In [155]:
#################################ALL LINES BELOW NO LONGER USED ############################################
#################################ALL LINES BELOW NO LONGER USED ############################################
#################################ALL LINES BELOW NO LONGER USED ############################################

filename='all_results_count_holdout.pkl'
results=pickle.load(open(filename,'rb'))
filename='all_results_count.pkl'
results_cv=pickle.load(open(filename,'rb'))
intervals=range(30,3601,30)
color={'smac':'r','hyperopt':'b','nvb':'y','random':'g', 'random_2x':'g','random_3x':'g','nvb_smac':'y','random_smac':'g','random_smac_2x':'g','random_smac_3x':'g','rf_baseline':'k'}
linestyle={'smac':'-','hyperopt':'-','nvb':'-','random':'-', 'random_2x':'-','random_3x':'-','nvb_smac':'--','random_smac':'--','random_smac_2x':'--','random_smac_3x':'--','rf_baseline':'-'}
marker={'random_2x':'+','random_3x':'x','random_smac_2x':'+','random_smac_3x':'x'}
    
pdf_file=PdfPages('seacher_counts')
searchers=results[results.keys()[0]].keys()
print searchers
searchers_count_avg={}
for s in searchers:
    counts=numpy.asarray([numpy.asarray(results[d][s][0]) for d in results.keys()])
    searchers_count_avg[s]=numpy.mean(counts,axis=0)
searchers_count_avg_cv={}
for s in searchers:
    counts=numpy.asarray([numpy.asarray(results_cv[d][s][0]) for d in results.keys()])
    searchers_count_avg_cv[s]=numpy.mean(counts,axis=0)
searchers_count_rel={}
for s in searchers:
    dataset_times=[]
    for t in range(len(intervals)):
        indices=[ n for n,i in enumerate(searchers_count_avg[s][0:len(intervals)]) if i<=searchers_count_avg_cv[s][t]]
        if len(indices)>0:
            dataset_times.append(intervals[indices[-1]]*1.0/intervals[t])
        else:
            dataset_times.append(1)
    searchers_count_rel[s]=dataset_times
for s in searchers:
    plt.plot(intervals,searchers_count_avg[s][0:len(intervals)],linestyle='-',color=color[s],label=s+' holdout')
    plt.plot(intervals,searchers_count_avg_cv[s][0:len(intervals)],linestyle='--',color=color[s],label=s+' cv')
    plt.plot(intervals,[numpy.max(searchers_count_avg_cv[s][0:len(intervals)])]*len(intervals),linestyle=':',color=color[s])

plt.legend(loc=2)
plt.xlabel('Time (s)')
plt.ylabel('# of Evals')
pdf_file.savefig()
plt.close()
s='nvb'
plt.plot(intervals,searchers_count_avg[s][0:len(intervals)],linestyle='-',color=color[s],label=s+' holdout')
plt.plot(intervals,searchers_count_avg_cv[s][0:len(intervals)],linestyle='--',color=color[s],label=s+' cv')
plt.plot(intervals,[numpy.max(searchers_count_avg_cv[s][0:len(intervals)])]*len(intervals),linestyle=':',color=color[s])
plt.ylim(0,20)
plt.legend(loc=2)
plt.xlabel('Time (s)')
plt.ylabel('# of Evals')
#plt.show()
pdf_file.savefig()
plt.close()
for s in searchers:
    plt.plot(intervals[10:len(searchers_count_rel[s]):5],searchers_count_rel[s][10::5],linestyle='-',color=color[s],label=s)
plt.legend(loc=2)
#plt.ylim([0,.2])
plt.xlabel('Time (s)')
plt.ylabel('Ratio of time for Holdout to reach same # of evals as CV')
pdf_file.savefig()
plt.close()


pdf_file.close()

['smac', 'hyperopt', 'random', 'nvb']


In [19]:
count_best={'smac':numpy.zeros(len(intervals)),'hyperopt':numpy.zeros(len(intervals)),'nvb':numpy.zeros(len(intervals)),'random':numpy.zeros(len(intervals))}
which_best={'smac':{},'hyperopt':{},'nvb':{},'random':{}}
datasets=results.keys()
datasets.remove(24)
for t in range(len(intervals)):
    for s in searchers:
        which_best[s][t]=[]
    for d in datasets:
        best_searcher=searchers[numpy.argmin([results[d][s][3][t] for s in searchers])]
        t_stat=[(results[d][s][3][t]-results[d][best_searcher][3][t])/numpy.sqrt(1/10.*(results[d][s][4][t]**2+results[d][best_searcher][4][t]**2)) for s in searchers if s != best_searcher]
        dfs = [(results[d][s][4][t]**2/10.+results[d][best_searcher][4][t]**2/10.)**2/((results[d][s][4][t]**2/10.)**2/9.+(results[d][best_searcher][4][t]**2/10.)**2/9.) for s in searchers if s != best_searcher]
        comp_to_best=[scipy.stats.t.cdf(t_stat[s],dfs[s]) > 0.95 for s in range(3)]
        if numpy.isnan(numpy.sum(comp_to_best)):
            print t,d
        
        if sum(comp_to_best)==3:
            count_best[best_searcher][t]+=1
            which_best[best_searcher][t].append(d)
        

NameError: name 'searchers' is not defined

In [132]:
d=59
best_searcher='random'
t=10
t_stat=[(results[d][s][3][t]-results[d][best_searcher][3][t])/numpy.sqrt(1/10.*(results[d][s][4][t]**2+results[d][best_searcher][4][t]**2)) for s in searchers if s != best_searcher]
dfs = [(results[d][s][4][t]**2/10.+results[d][best_searcher][4][t]**2/10.)**2/((results[d][s][4][t]**2/10.)**2/9.+(results[d][best_searcher][4][t]**2/10.)**2/9.) for s in searchers if s != best_searcher]
comp_to_best=[scipy.stats.t.cdf(t_stat[s],dfs[s]) > 0.95 for s in range(3)]

In [15]:
combined_results[2]['rf_baseline']

[10, 0.0, 0.0, 1.2499199999999999, 2.4998399999999998, 0.0, 0.0]

In [168]:
#pdf_file=PdfPages('aggregate_errors.pdf')
color={'smac':'r','hyperopt':'b','nvb':'y','random':'g'}
plt.figure(figsize=(11,8.5))
for s in searchers:
    plt.plot(intervals,count_best[s][0:len(intervals)],label=s,color=color[s])
plt.xlabel('Time (s)')
plt.legend(loc=1)
plt.show()

In [190]:
array=numpy.array([1,1,3,4])
temp = array.argsort()
ranks = numpy.empty(len(array), int)
ranks[temp] = numpy.arange(len(array))

In [222]:
searchers_val_avg.keys()

['random_3x',
 'random_2x_smac',
 'hyperopt',
 'random',
 'nvb_smac',
 'smac',
 'nvb',
 'random_smac',
 'random_3x_smac',
 'random_2x']

In [176]:
range(4,120,5)

[4,
 9,
 14,
 19,
 24,
 29,
 34,
 39,
 44,
 49,
 54,
 59,
 64,
 69,
 74,
 79,
 84,
 89,
 94,
 99,
 104,
 109,
 114,
 119]